In [2]:
import geopandas as gpd
import rasterio
from rasterstats import zonal_stats
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [2]:
shp_path = "data/shapes/distritos.shp"
gdf = gpd.read_file(shp_path)

In [3]:
gdf

,IDDPTO,DEPARTAMEN,IDPROV,PROVINCIA,IDDIST,DISTRITO,CAPITAL,CODCCPP,AREA,FUENTE,geometry
0,10,HUANUCO,1009,PUERTO INCA,100902,CODO DEL POZUZO,CODO DEL POZUZO,0001,1,INEI,"POLYGON ((-75.31797 -9.29529, -75.3171 -9.2975..."
1,10,HUANUCO,1009,PUERTO INCA,100904,TOURNAVISTA,TOURNAVISTA,0001,1,INEI,"POLYGON ((-74.64136 -8.82302, -74.64036 -8.828..."
2,25,UCAYALI,2503,PADRE ABAD,250305,ALEXANDER VON HUMBOLDT,ALEXANDER VON HUMBOLDT,0001,1,INEI,"POLYGON ((-75.02253 -8.74193, -75.02267 -8.742..."
3,25,UCAYALI,2503,PADRE ABAD,250302,IRAZOLA,SAN ALEJANDRO,0001,1,INEI,"POLYGON ((-75.13864 -8.56712, -75.13956 -8.569..."
4,25,UCAYALI,2503,PADRE ABAD,250304,NESHUYA,MONTE ALEGRE,0001,1,INEI,"POLYGON ((-75.01589 -8.44637, -75.01585 -8.446..."
...,...,...,...,...,...,...,...,...,...,...,...
1868,10,HUANUCO,1006,LEONCIO PRADO,100608,CASTILLO GRANDE,CASTILLO GRANDE,0001,1,INEI,"POLYGON ((-76.08083 -9.13017, -76.08026 -9.130..."
1869,10,HUANUCO,1006,LEONCIO PRADO,100609,PUEBLO NUEVO,PUEBLO NUEVO,0001,1,INEI,"POLYGON ((-75.88828 -9.00906, -75.88756 -9.010..."
1870,10,HUANUCO,1006,LEONCIO PRADO,100610,SANTO DOMINGO DE ANDA,PACAE,0001,1,INEI,"POLYGON ((-75.91141 -8.88593, -75.91182 -8.886..."
1871,21,PUNO,2111,SAN ROMAN,211105,SAN MIGUEL,SAN MIGUEL,0001,1,INEI,"POLYGON ((-70.13203 -15.33382, -70.12355 -15.3..."


In [4]:
gdf = gdf.rename(columns={"IDDIST": "UBIGEO"})


In [5]:
gdf["UBIGEO"] = gdf["UBIGEO"].astype(str).str.strip()

In [6]:
# Normalizamos nombres (sin tildes y en mayúscula)
if "NOMBRE" in gdf.columns:
    gdf["NOMBRE"] = (
        gdf["NOMBRE"]
        .str.upper()
        .str.normalize("NFKD")
        .str.encode("ascii", errors="ignore")
        .str.decode("utf-8")
    )


In [7]:
raster_path = "data/tmin_raster.tif"
src = rasterio.open(raster_path)

print(f"Raster CRS: {src.crs}, Bounds: {src.bounds}, Bands: {src.count}")

Raster CRS: EPSG:4326, Bounds: BoundingBox(left=-81.38, bottom=-18.65000000000002, right=-67.1300000000008, top=1.2000000000000002), Bands: 5


In [8]:
gdf = gdf.loc[:, ~gdf.columns.duplicated()]

In [9]:
# Función de estadísticas zonales

def compute_stats(band=1):
    affine = src.transform
    array = src.read(band)

    stats = zonal_stats(
        vectors=gdf,
        raster=array,
        affine=affine,
        stats=["count", "mean", "min", "max", "std"],
        add_stats={
            "p10": lambda x: np.nanpercentile(np.array(x, copy=True), 10) if len(x) > 0 else np.nan,
            "p90": lambda x: np.nanpercentile(np.array(x, copy=True), 90) if len(x) > 0 else np.nan,
            "range": lambda x: (np.nanmax(x) - np.nanmin(x)) if len(x) > 0 else np.nan,
        },
        nodata=src.nodata
    )

    df = pd.DataFrame(stats)
    df["BAND"] = band
    df["YEAR"] = 2019 + band  # Ejemplo: Band 1 = 2020
    return df


# Iteramos sobre todas las bandas

all_stats = []
for b in range(1, src.count + 1):
    df_band = compute_stats(band=b)
    all_stats.append(df_band)

# Concatenar todas las bandas
stats_df = pd.concat(all_stats, axis=0).reset_index(drop=True)

# Prefijo a columnas
stats_df = stats_df.add_prefix("TMIN_")

# Repetir geometrías según número de bandas
gdf_repeated = pd.concat([gdf.reset_index(drop=True)] * src.count, ignore_index=True)

# Unir shapefile + stats
result = pd.concat([gdf_repeated, stats_df], axis=1)


C:\Users\Joaquin\anaconda3\envs\tarea_3\Lib\site-packages\rasterstats\io.py:335: NodataWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn(


In [10]:
gdf = gdf.loc[:, ~gdf.columns.duplicated()]

In [ ]:
# Guardamos resultados

# GeoJSON para mapas dinámicos
result.to_file("data/tmin_stats.geojson", driver="GeoJSON")

# CSV para tablas
result.drop(columns="geometry").to_csv("data/tmin_stats.csv", index=False)


In [ ]:

# Visualizaciones

# Histograma distribución de Tmin media distrital
plt.figure(figsize=(8, 5))
result["TMIN_mean"].hist(bins=30)
plt.title("Distribución de Tmin Media por Distrito")
plt.xlabel("Tmin (°C)")
plt.ylabel("Frecuencia")
plt.savefig("data/outputs/distribucion_tmin.png", dpi=150)
plt.show()

In [ ]:
# Ranking top 15 distritos más fríos (ejemplo para último año disponible)
latest_year = result["TMIN_YEAR"].max()
ranking = (
    result[result["TMIN_YEAR"] == latest_year]
    .sort_values("TMIN_mean")
    .head(15)
)

plt.figure(figsize=(10, 6))
plt.barh(ranking["DISTRITO"], ranking["TMIN_mean"], color="steelblue")
plt.title(f"Top 15 distritos más fríos - Año {latest_year}")
plt.xlabel("Tmin (°C)")
plt.gca().invert_yaxis()
plt.savefig("data/outputs/top15_frio.png", dpi=150)
plt.show()


In [ ]:
latest_year = result["TMIN_YEAR"].max()

ranking_hot = (
    result[result["TMIN_YEAR"] == latest_year]
    .sort_values("TMIN_mean", ascending=False)  # ordenar de mayor a menor
    .head(15)
)

plt.figure(figsize=(10, 6))
plt.barh(ranking_hot["DISTRITO"], ranking_hot["TMIN_mean"], color="darkorange")
plt.title(f"Top 15 distritos más calientes - Año {latest_year}")
plt.xlabel("Tmin (°C)")
plt.gca().invert_yaxis()
plt.savefig("data/outputs/top15_caliente.png", dpi=150)
plt.show()


In [ ]:
# Mapa estático (ejemplo para último año)
fig, ax = plt.subplots(figsize=(8, 10))
(
    result[result["TMIN_YEAR"] == latest_year]
    .plot(column="TMIN_mean", cmap="coolwarm", legend=True, ax=ax)
)
plt.title(f"Temperatura Mínima Media por Distrito - Año {latest_year}")
plt.axis("off")
plt.savefig("data/outputs/mapa_tmin.png", dpi=150)
plt.show()

In [5]:
t_min_stats = pd.read_csv("Data/tmin_stats.csv")
t_min_stats.head()

,IDDPTO,DEPARTAMEN,IDPROV,PROVINCIA,UBIGEO,DISTRITO,CAPITAL,CODCCPP,AREA,FUENTE,TMIN_min,TMIN_max,TMIN_mean,TMIN_count,TMIN_std,TMIN_p10,TMIN_p90,TMIN_range,TMIN_BAND,TMIN_YEAR
0,10,HUANUCO,1009,PUERTO INCA,100902,CODO DEL POZUZO,CODO DEL POZUZO,1,1,INEI,8.903979,22.604116,18.932154,107,3.430219,13.637902,22.388206,13.700137,1,2020
1,10,HUANUCO,1009,PUERTO INCA,100904,TOURNAVISTA,TOURNAVISTA,1,1,INEI,19.775057,22.860834,22.290649,57,0.464426,21.846657,22.536060,3.0857773,1,2020
2,25,UCAYALI,2503,PADRE ABAD,250305,ALEXANDER VON HUMBOLDT,ALEXANDER VON HUMBOLDT,1,1,INEI,21.953405,22.141968,22.056366,6,0.076649,21.958242,22.266325,0.1885624,1,2020
3,25,UCAYALI,2503,PADRE ABAD,250302,IRAZOLA,SAN ALEJANDRO,1,1,INEI,21.661476,22.481043,22.263211,62,0.158932,22.072361,22.475220,0.8195667,1,2020
4,25,UCAYALI,2503,PADRE ABAD,250304,NESHUYA,MONTE ALEGRE,1,1,INEI,21.975763,22.385101,22.173068,22,0.118299,22.025274,22.388065,0.409338,1,2020


In [32]:

# Calculamos el percentil 10 global de TMIN_mean
global_p10 = t_min_stats["TMIN_mean"].quantile(0.10)

# Filtramos distritos cuyo TMIN_mean es menor o igual a ese valor
target = t_min_stats[t_min_stats["TMIN_mean"] <= global_p10]

# Mostramos columnas clave
target_selection = target[["DEPARTAMEN", "PROVINCIA", "DISTRITO", "UBIGEO", "TMIN_mean"]]

target_unique = target_selection.groupby(["UBIGEO", "DEPARTAMEN", "PROVINCIA", "DISTRITO"], as_index=False)["TMIN_mean"].min()
print(target_unique)


     UBIGEO DEPARTAMEN  PROVINCIA              DISTRITO  TMIN_mean
0     30305   APURIMAC  ANTABAMBA               OROPESA   0.365621
1     30702   APURIMAC       GRAU            CURPAHUASI   0.938246
2     30713   APURIMAC       GRAU               VIRUNDO   1.070473
3     40103   AREQUIPA   AREQUIPA                 CAYMA  -0.487927
4     40119   AREQUIPA   AREQUIPA  SAN JUAN DE TARUCANI  -3.747299
..      ...        ...        ...                   ...        ...
191  230203      TACNA  CANDARAVE              CAMILACA  -2.636052
192  230401      TACNA     TARATA                TARATA  -5.533942
193  230406      TACNA     TARATA              SUSAPAYA  -5.989168
194  230407      TACNA     TARATA             TARUCACHI   0.416256
195  230408      TACNA     TARATA                TICACO  -5.011880

[196 rows x 5 columns]


In [33]:
# Filtramos solo los nombres únicos de los departamentos
departamentos = target_unique["DEPARTAMEN"].unique()

#algunas estadísticas:

print("Departamentos en el percentil 10 de Tmin:")
for dpto in departamentos:
    print("-", dpto)
    
# Calculamos promedio simple de TMIN_mean entre los distritos en target_unique
promedio_tmin = target_unique["TMIN_mean"].mean()

print("Promedio de TMIN_mean en distritos target:", promedio_tmin)

#calculamos promedio por departamento:

promedio_por_depto = target_unique.groupby("DEPARTAMEN")["TMIN_mean"].mean()

print("Promedio de TMIN_mean por departamento (distritos target):")
print(promedio_por_depto)



Departamentos en el percentil 10 de Tmin:
- APURIMAC
- AREQUIPA
- AYACUCHO
- CUSCO
- HUANCAVELICA
- JUNIN
- LIMA
- MOQUEGUA
- PASCO
- PUNO
- TACNA
Promedio de TMIN_mean en distritos target: -0.8898835354047264
Promedio de TMIN_mean por departamento (distritos target):
DEPARTAMEN
APURIMAC        0.791447
AREQUIPA       -1.488453
AYACUCHO        0.745413
CUSCO          -0.951615
HUANCAVELICA   -0.138641
JUNIN          -0.087548
LIMA            0.235217
MOQUEGUA       -1.415439
PASCO           0.949843
PUNO           -0.979028
TACNA          -3.378845
Name: TMIN_mean, dtype: float64


In [26]:
# Definimos departamentos amazónicos
amazon_depts = ["LORETO", "UCAYALI", "MADRE DE DIOS"]

# Filtramos solo esos departamentos
t_min_stats_amazonicos = t_min_stats[t_min_stats["DEPARTAMEN"].str.upper().isin(amazon_depts)]

# Calculamos el percentil 10 global de TMIN_mean (en todo el país)
global_p10_amazonicos = t_min_stats_amazonicos["TMIN_mean"].quantile(0.10)

# Filtrar distritos que están en el p10 global de departamentos amazónicos
amazon_target = t_min_stats_amazonicos[
    (t_min_stats_amazonicos["TMIN_mean"] <= global_p10_amazonicos)
]

# Seleccionar columnas clave
amazon_target_selection = amazon_target[["DEPARTAMEN", "PROVINCIA", "DISTRITO", "UBIGEO", "TMIN_mean"]]

target_unique_amazonicos = amazon_target_selection.groupby(["UBIGEO", "DEPARTAMEN", "PROVINCIA", "DISTRITO"], as_index=False)["TMIN_mean"].min()
print(target_unique_amazonicos)


    UBIGEO     DEPARTAMEN          PROVINCIA       DISTRITO  TMIN_mean
0   160202         LORETO      ALTO AMAZONAS    BALSAPUERTO  20.705258
1   160601         LORETO            UCAYALI      CONTAMANA  20.808713
2   160604         LORETO            UCAYALI  PAMPA HERMOSA  20.176199
3   160701         LORETO  DATEM DEL MARAÑON       BARRANCA  20.932087
4   160703         LORETO  DATEM DEL MARAÑON     MANSERICHE  21.465507
5   170201  MADRE DE DIOS               MANU           MANU  19.133363
6   170203  MADRE DE DIOS               MANU  MADRE DE DIOS  21.390409
7   170204  MADRE DE DIOS               MANU      HUEPETUHE  20.623789
8   170302  MADRE DE DIOS          TAHUAMANU         IBERIA  21.460159
9   250103        UCAYALI   CORONEL PORTILLO         IPARIA  21.318644
10  250201        UCAYALI            ATALAYA       RAIMONDI  21.023004
11  250301        UCAYALI         PADRE ABAD     PADRE ABAD  19.767534


In [36]:
# Imprimir solo los nombres únicos de los departamentos
departamentos = target_unique_amazonicos["DEPARTAMEN"].unique()

#algunas estadísticas:

print("Departamentos en el percentil 10 de Tmin:")
for dpto in departamentos:
    print("-", dpto)
    
# Calcular promedio simple de TMIN_mean entre los distritos en target_unique
promedio_tmin_amazonicos_p10 = target_unique_amazonicos["TMIN_mean"].mean()

print("Promedio de TMIN_mean en distritos target amazónicos:", promedio_tmin)

# calculamos promedio por departamento:

promedio_por_depto_amazonicos = target_unique_amazonicos.groupby("DEPARTAMEN")["TMIN_mean"].mean()

print("Promedio de TMIN_mean por departamento (distritos target amazonicos):")
print(promedio_por_depto_amazonicos)


Departamentos en el percentil 10 de Tmin:
- LORETO
- MADRE DE DIOS
- UCAYALI
Promedio de TMIN_mean en distritos target amazónicos: -0.8898835354047264
Promedio de TMIN_mean por departamento (distritos target amazonicos):
DEPARTAMEN
LORETO           20.817553
MADRE DE DIOS    20.651930
UCAYALI          20.703061
Name: TMIN_mean, dtype: float64
